In [1]:
# Import the models and functions
import torch
import os.path
from models import *
from trainer import *
import matplotlib.pyplot as plt
from torchinfo import summary


In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np

# Load in data, this assumed you have a folder in env named data
data_dir = "data"
X_test = np.load(os.path.join(data_dir, "X_test.npy"))
y_test = np.load(os.path.join(data_dir, "y_test.npy"))
person_test = np.load(os.path.join(data_dir, "person_test.npy")).squeeze(axis=1)
X_train_valid = np.load(os.path.join(data_dir, "X_train_valid.npy"))
y_train_valid = np.load(os.path.join(data_dir, "y_train_valid.npy"))
person_train_valid = np.load(os.path.join(data_dir, "person_train_valid.npy")).squeeze(axis=1)

# Predefine some useful variables and fix data a bit
n_class = len(set(y_train_valid))
n_trials = 5
min_y = min(y_train_valid)
y_train_valid = y_train_valid - min_y
y_test = y_test - min_y

# Set up data sets
s1_indices_train = [i for i, s in enumerate(person_train_valid) if s == 0]
s1_indices_test = [i for i, s in enumerate(person_test) if s == 0]
X_train_valid_s1 = X_train_valid[s1_indices_train,:,:400]
y_train_valid_s1 = y_train_valid[s1_indices_train]
X_test_s1 = X_test[s1_indices_test,:,:400]
y_test_s1 = y_test[s1_indices_test]

# Make dataloader test set for the single subject
# Convert data to tensors
X_tensor = torch.FloatTensor(X_test_s1)
y_tensor = torch.LongTensor(y_test_s1)

# Combine X and y into a TensorDataset
dataset = TensorDataset(X_tensor, y_tensor)

# Prepare dataloaders
test_dataloader_s1 = DataLoader(dataset, batch_size=256, shuffle=False)

In [3]:
# Tonmoy architecture
Tonmoy_summary = HybridCNNLSTM()
summary(Tonmoy_summary, input_size=(64, 22, 1000))

/Users/turnerkaminski/anaconda3/lib/python3.10/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.4 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Layer (type:depth-idx)                   Output Shape              Param #
HybridCNNLSTM                            [64, 4]                   --
├─Sequential: 1-1                        [64, 25, 18, 332]         --
│    └─Conv2d: 2-1                       [64, 25, 18, 996]         650
│    └─ELU: 2-2                          [64, 25, 18, 996]         --
│    └─MaxPool2d: 2-3                    [64, 25, 18, 332]         --
│    └─BatchNorm2d: 2-4                  [64, 25, 18, 332]         50
│    └─Dropout: 2-5                      [64, 25, 18, 332]         --
├─Sequential: 1-2                        [64, 50, 14, 109]         --
│    └─Conv2d: 2-6                       [64, 50, 14, 328]         31,300
│    └─ELU: 2-7                          [64, 50, 14, 328]         --
│    └─MaxPool2d: 2-8                    [64, 50, 14, 109]         --
│    └─BatchNorm2d: 2-9                  [64, 50, 14, 109]         100
│    └─Dropout: 2-10                     [64, 50, 14, 109]         --
├─Sequent

In [4]:

# EEGNet architecture
EEGnet_summary = CNN(input_size=X_train_valid.shape[1:], N=n_class)
summary(EEGnet_summary, input_size=(64, 22, 1000))

Layer (type:depth-idx)                   Output Shape              Param #
CNN                                      [64, 4]                   --
├─Sequential: 1-1                        [64, 8, 22, 969]          --
│    └─Conv2d: 2-1                       [64, 8, 22, 969]          256
│    └─BatchNorm2d: 2-2                  [64, 8, 22, 969]          16
├─Sequential: 1-2                        [64, 16, 1, 969]          --
│    └─Conv2d: 2-3                       [64, 16, 1, 969]          352
│    └─BatchNorm2d: 2-4                  [64, 16, 1, 969]          32
│    └─ELU: 2-5                          [64, 16, 1, 969]          --
├─AvgPool2d: 1-3                         [64, 16, 1, 484]          --
├─Dropout: 1-4                           [64, 16, 1, 484]          --
├─Sequential: 1-5                        [64, 16, 1, 119]          --
│    └─Conv2d: 2-6                       [64, 16, 1, 477]          2,048
│    └─Conv2d: 2-7                       [64, 16, 1, 477]          256
│    └─Ba

In [5]:

# LSTM architecture
lstm_summary = LSTM()
summary(lstm_summary, input_size=(64, 22, 1000))

Layer (type:depth-idx)                   Output Shape              Param #
LSTM                                     [64, 4]                   --
├─LSTM: 1-1                              [64, 1000, 64]            89,088
├─Sequential: 1-2                        [64, 4]                   --
│    └─Linear: 2-1                       [64, 54]                  3,510
│    └─BatchNorm1d: 2-2                  [64, 54]                  108
│    └─ReLU: 2-3                         [64, 54]                  --
│    └─Dropout: 2-4                      [64, 54]                  --
│    └─Linear: 2-5                       [64, 44]                  2,420
│    └─BatchNorm1d: 2-6                  [64, 44]                  88
│    └─ReLU: 2-7                         [64, 44]                  --
│    └─Linear: 2-8                       [64, 24]                  1,080
│    └─BatchNorm1d: 2-9                  [64, 24]                  48
│    └─ReLU: 2-10                        [64, 24]                  --
│

In [6]:
# LSTM architecture
mini_summary = small_CRNN()
summary(mini_summary, input_size=(64, 22, 1000))

/Users/turnerkaminski/anaconda3/lib/python3.10/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.4 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: [Sequential: 1, Conv2d: 2, ReLU: 2, BatchNorm2d: 2, Dropout: 2, Sequential: 1, Conv2d: 2, ReLU: 2, BatchNorm2d: 2, Dropout: 2]